In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import RNNModel
from darts.metrics import mse, rmse, r2_score, mae, smape
from darts.dataprocessing.transformers import Scaler
import torch
from pytorch_lightning.callbacks import Callback, EarlyStopping
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Configurações de visualização
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
torch.set_float32_matmul_precision('medium')

In [2]:
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

df = pd.read_csv("../data/ground_station_clean.csv")

In [3]:
encoders = {
    "cyclic":{
        "future":["month","dayofyear","day","hour","minute"],
    },
    "transformer": Scaler()
}

In [4]:
class PatchedPruningCallback(PyTorchLightningPruningCallback, Callback):
    pass

# Criar diretório para salvar os resultados das iterações
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)

In [5]:
def objective(trial):

    input_chunk_length = trial.suggest_int("input_chunk_length", 12, 72)
    training_length = trial.suggest_int("training_length", input_chunk_length + 1, input_chunk_length + 24 if input_chunk_length + 24 > input_chunk_length + 1 else input_chunk_length + 2) # Garante que o limite superior seja maior que o inferior
    hidden_size = trial.suggest_int("hidden_size", 32, 256)
    layer_size = trial.suggest_int("layer_size", 1, 4)
    dropout = trial.suggest_float("dropout", 0.1, 0.4)
    batch_size = trial.suggest_int("batch_size", 16, 64)
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    forecast_horizon = trial.suggest_int("forecast_horizon", 1, 24)

    early_stopper = EarlyStopping("val_loss", min_delta=0.001, patience=10, verbose=True)
    pl_trainer_kwargs = {
            "accelerator": "auto",
            "callbacks": [early_stopper],
    }
    n = int(len(df)*0.8)
    train_df_fold, val_df_fold = df.iloc[:n], df.iloc[n:]
    train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
    val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

    print(f"\nIniciando Trial {trial.number}")
    print(f"Hiperparâmetros: {trial.params}")
    print("\nTreinando o modelo...")
    print(f"Train set: {len(train_fold)} samples") # Usar len() para TimeSeries
    print(f"Validation set: {len(val_fold)} samples") # Usar len() para TimeSeries
    
    scaler = Scaler()
    scaler = scaler.fit(train_fold)
    train_scaled = scaler.transform(train_fold)
    val_scaled = scaler.transform(val_fold)

    # Definir work_dir e model_name para consistência
    # force_reset=True irá limpar work_dir/model_name antes de cada treinamento da iteração.
    _work_dir = "/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs/" # Certifique-se que este caminho existe e é gravável
    _model_name = "model_optuna_temp" # Nome do modelo temporário para cada iteração

    # Criar o diretório _work_dir se não existir
    os.makedirs(_work_dir, exist_ok=True)

    model = RNNModel(
        model="LSTM",
        model_name=_model_name,
        work_dir=_work_dir,
        input_chunk_length=input_chunk_length,
        hidden_dim=hidden_size,
        n_rnn_layers=layer_size,
        n_epochs=30, # Considere tornar n_epochs um hiperparâmetro ou usar um valor maior com EarlyStopping
        batch_size=batch_size,
        dropout=dropout,
        add_encoders=encoders,
        training_length=training_length,
        pl_trainer_kwargs=pl_trainer_kwargs,
        loss_fn=torch.nn.HuberLoss(),
        lr_scheduler_cls=torch.optim.lr_scheduler.ReduceLROnPlateau,
        lr_scheduler_kwargs={"mode":"min", "factor":0.5, "patience":4, "min_lr":1e-6},
        save_checkpoints=True,    
        show_warnings=True,
        force_reset=True, # Importante para que cada iteração treine do zero com este model_name
        optimizer_kwargs={"lr": lr, "weight_decay": 1e-5},
    )
    model.fit(
        series=train_scaled,
        val_series=val_scaled,
        verbose=True # Pode ser False para menos output durante a otimização do Optuna
    )
    
    try:
        loaded_model = RNNModel.load_from_checkpoint(model_name=_model_name, work_dir=_work_dir, best=True)
        print(f"Modelo carregado do checkpoint para trial {trial.number}")
    except FileNotFoundError:
        print(f"Checkpoint não encontrado para {_model_name} em {_work_dir}. Usando o modelo treinado em memória.")
        loaded_model = model # Usar o modelo treinado se o carregamento falhar (fallback)


    forecasts = loaded_model.historical_forecasts(
        train_scaled,# Como no script original
        start=0.8, # Início da previsão no conjunto de treinamento
        forecast_horizon=forecast_horizon,
        stride=1, # Gera uma previsão em cada ponto de tempo possível
        retrain=False,
        verbose=True # Pode ser False
    ) 

    forecasts_t = scaler.inverse_transform(forecasts)
    s = scaler.inverse_transform(train_scaled).split_after(0.8)[1]
    metrics = {}
    print("Iniciando a verificação das séries temporais para constância...")
    try:
        for target in target_columns:
            metrics[target] = {
                'MSE': mse(s[target], forecasts_t[target]),
                'RMSE': rmse(s[target], forecasts_t[target]),
                'MAE': mae(s[target], forecasts_t[target]),
                'R2': r2_score(s[target], forecasts_t[target]),
                'SMAPE': smape(s[target], forecasts_t[target]),
            }
        metrics_df = pd.DataFrame(metrics).T
        print("\nMétricas de performance:")
        print(metrics_df)
    except Exception as e:
        print(e)

    overall_smape_val = smape(s, forecasts_t)
    print(f"O SMAPE desse fold foi {overall_smape_val}")

    trial_dict = {
        "trial_number": trial.number,
        "input_chunk_length": input_chunk_length,
        "training_length": training_length,
        "hidden_size": hidden_size,
        "layer_size": layer_size,
        "dropout": dropout,
        "batch_size": batch_size,
        "lr": lr,
        "forecast_horizon": forecast_horizon,
        "overall_smape_val": overall_smape_val,
        "metrics": metrics_df.to_dict(),
    }
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Resultados do trial {trial.number} salvos em {json_path}")
    return overall_smape_val if not np.isnan(overall_smape_val) else float("inf")



In [6]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [7]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50, callbacks=[print_callback])

[I 2025-05-13 16:49:35,249] A new study created in memory with name: no-name-0b9623e9-b403-48bf-afcb-87568b404491



Iniciando Trial 0
Hiperparâmetros: {'input_chunk_length': 40, 'training_length': 53, 'hidden_size': 129, 'layer_size': 2, 'dropout': 0.26674465690069815, 'batch_size': 48, 'lr': 0.0023072805979340964, 'forecast_horizon': 24}

Treinando o modelo...
Train set: 6647 samples
Validation set: 1662 samples


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | HuberLoss        | 0      | train
1 | train_criterion | HuberLoss        | 0      | train
2 | val_criterion   | HuberLoss        | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | LSTM             | 211 K  | train
6 | V               | Linear           | 1.0 K  | train
-------------------------------------------------------------
212 K     Trainable params
0         Non-trainable params
212 K     Total params
0.848     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


...

Modelo carregado do checkpoint para trial 49


/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

[I 2025-05-13 19:22:47,761] Trial 49 finished with value: 46.31042367328655 and parameters: {'input_chunk_length': 66, 'training_length': 88, 'hidden_size': 154, 'layer_size': 2, 'dropout': 0.2035086405960666, 'batch_size': 27, 'lr': 2.8855969357087103e-05, 'forecast_horizon': 2}. Best is trial 12 with value: 42.009663079630656.


Iniciando a verificação das séries temporais para constância...

Métricas de performance:
                                    MSE        RMSE        MAE         R2  \
Temperature                    0.694993    0.833663   0.610446   0.795851   
Precipitation_accumulated     26.131155    5.111864   3.252116 -26.531298   
Humidity                      10.975485    3.312927   2.576255   0.831221   
Wind_Speed_kmh                 0.109207    0.330465   0.136763   0.679094   
Soil_Moisture              40822.863293  202.046686  84.126346   0.944832   
Soil_Temperature               0.707200    0.840952   0.628792   0.873989   
Wind_Dir_Sin                   0.060703    0.246379   0.119097   0.622071   
Wind_Dir_Cos                   0.055937    0.236509   0.114927   0.805162   

                                SMAPE  
Temperature                 10.610084  
Precipitation_accumulated    0.704185  
Humidity                     3.087119  
Wind_Speed_kmh             154.539914  
Soil_Moisture   

In [8]:
print(f"Melhor Valor (Mínimo): {study.best_value}")
print(f"Melhores Parâmetros: {study.best_params}")

Melhor Valor (Mínimo): 42.009663079630656
Melhores Parâmetros: {'input_chunk_length': 60, 'training_length': 82, 'hidden_size': 212, 'layer_size': 1, 'dropout': 0.39653758740344514, 'batch_size': 18, 'lr': 3.0378489033878987e-05, 'forecast_horizon': 1}


In [9]:
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Melhores resultados salvos em {json_path}")

Melhores resultados salvos em optuna_iteration_metrics/best_trial.json
